In [11]:
import bqplot
import os
from bqplot_image_gl import ImageGL
from astropy.io import fits
from astropy.visualization import PercentileInterval
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot import pyplot as plt
from bqplot import Toolbar
import numpy as np
import ipyvuetify as v
from ipywidgets import Layout
import ipywidgets as widgets
import traitlets


In [12]:
# this is a widget for debugging
out = widgets.Output()
out.add_traits(_metadata=traitlets.Dict(default_value={'mount_id': 'out'}).tag(sync=True))
out

Output()

In [13]:
jwst_sources = [227, 482, 546, 1186]
selected_src = jwst_sources[np.random.randint(0,3)]
def prep_data(source):
    ''' prepare test data '''
    path = '/Users/bcherinka/Work/mosviz/data'
    path1d = os.path.join(path, 'jwst_level3_NIRSpec/jw95065_{0}_nrs_msaspec_barshadow_x1d.fits'.format(source))
    path2d = os.path.join(path, 'jwst_level3_NIRSpec/jw95065_{0}_nrs_msaspec_barshadow_s2d.fits'.format(source))
    pathimg = os.path.join(path, 'jwst_level3_NIRSpec/cutouts/SOURCEID_{0}.fits'.format(source))
    hdu1d = fits.open(path1d)
    hdu2d = fits.open(path2d)
    cut = fits.open(pathimg)
    return hdu1d, hdu2d, cut

In [14]:
hdu1d, hdu, cut = prep_data(selected_src)

In [15]:
hdu1d.info()

In [16]:
hdu1d[1].data

In [17]:
def get_xy(data):
    ''' get the xy data '''
    wave = data['WAVELENGTH']
    flux = data['FLUX']
    return wave, flux
wave, flux = get_xy(hdu1d[1].data)

In [18]:
 hdu.info()

In [19]:
def create_spec1d(x, y):
    ''' create a bqplot line chart for a 1d spectrum '''
    fig = plt.figure('Spectrum 1d')
    p = plt.plot(x, y)
    fig.layout.width = 'auto'
    fig.layout.height = 'auto'
    fig.layout.min_height = '400px' # so it shows nicely in the notebook
    return fig, p

In [20]:
fig, plot = create_spec1d(wave, flux)
fig

NameError: name 'wave' is not defined

In [17]:
def create_spec2d_heatmap(data):
    ''' create a 2d spectrum view as a bqplot Heatmap'''
    # trim the data to 95% of values
    i = PercentileInterval(95)
    limdata = i(data)

    # create the heatmap
    spec2d = plt.figure(padding_y=0)
    plt.scales(scales={'color': ColorScale(scheme='Greys', reverse=True)})
    axes_options = {'color': {'visible': False}}
    heat = plt.heatmap(limdata, axes_options=axes_options)
    return spec2d, heat

In [18]:
def create_spec2d_image(data):
    ''' create a 2d spectrum view as a bqplot ImageGL'''
    scale_x = LinearScale()
    scale_y = LinearScale()
    scales = {'x': scale_x,
              'y': scale_y}
    axis_x = Axis(scale=scale_x, label='x')
    axis_y = Axis(scale=scale_y, label='y', orientation='vertical')

    spec2d = Figure(scales=scales, axes=[axis_x, axis_y])

    scales_image = {'x': scale_x,
                    'y': scale_y,
                    'image': ColorScale(scheme='viridis', min=0, max=3)}

    image = ImageGL(image=hdu[1].data, scales=scales_image)

    spec2d.marks = (image,)
    return spec2d

In [19]:
# 2d spectra image as heatmap
spec2d, heat = create_spec2d_heatmap(hdu[1].data)
spec2d

Figure(axes=[ColorAxis(scale=ColorScale(reverse=True, scheme='Greys'), visible=False), Axis(scale=LinearScale(…

In [26]:
def create_cutout(data, target=None):
    ''' create image cutout as bqplot Heatmap '''
    aspect_ratio = data.shape[1] / data.shape[0]
    img = plt.figure(title='Src {0}'.format(target), layout=Layout(width='500px', height='500px'),
                     min_aspect_ratio=aspect_ratio, 
                     max_aspect_ratio=aspect_ratio, padding_y=0)
    plt.scales(scales={'color': ColorScale(scheme='Greys', reverse=True)})
    axes_options = {'x': {'visible': False}, 'y': {'visible': False}, 'color': {'visible': False}}
    p = plt.heatmap(data, axes_options=axes_options)
    return img, p

In [27]:
img, image = create_cutout(cut[0].data, target=selected_src)
img

Figure(axes=[ColorAxis(scale=ColorScale(reverse=True, scheme='Greys'), visible=False), Axis(scale=LinearScale(…

In [4]:
@out.capture()
def update_data(widget, event, data):
    #selected_src = jwst_sources[np.random.randint(0,3)]
    # get new data from selected target
    selected_src = int(data)
    hdu1d, hdu, cut = prep_data(selected_src)

    # update spec1d data
    wave, flux = get_xy(hdu1d[1].data)
    plot.y = flux
    plot.x = wave

    # update spec2d data
    i = PercentileInterval(95)
    heat.color = i(hdu[1].data)
    print('selected source', selected_src)
    print('old image title', img.title)
    # update image cutout data
    image.color = cut[0].data
    img.title = 'Src {0}'.format(selected_src)
    print('new image title', img.title)


In [73]:
# this class is needed to allow front-end defined components to send events to notebook defined components
# class UpdateRow(v.Row):
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
#         self.on_event('change', update_data)    

In [74]:
# create the Vuetify layout using v.Row and v.Col (Vuetify API v2.0 spec - replaces v.Layout and v.Flex, respectively)
row = v.Row(_metadata={'mount_id': 'protospec'}, dense=True, row=True, wrap=True, align_center=True, children=[

    # load the histogram and slider content
    v.Col(xs12=True, lg6=True, xl4=True, children=[
        img
    ]),

    # load the line plot content
    v.Col(xs12=True, xl4=True, children=[
        spec2d
    ]),
    # load the line plot content
    v.Col(xs12=True, xl4=True, children=[
        fig
    ]),       
])
# attach a change event so the Layout to update the data of all three components
row.on_event('change', update_data)

UpdateRow(children=[Col(children=[Figure(axes=[ColorAxis(scale=ColorScale(reverse=True, scheme='Greys'), visib…

In [31]:
# create an app bar to load a select button

#btn1 = v.Btn(color='primary', children=['Choose Data'])
#btn1.on_event('click', update_data)

btn1 = v.Select(dense=True, outlined=True, label='Target', items=jwst_sources)
btn1.on_event('change', update_data)
v.AppBar(_metadata={'mount_id': 'appbar'}, app=True, absolute=True, children=[
    v.Col(class_='col-md-2 col-lg-2', children=[btn1])
])

AppBar(absolute=True, app=True, children=[Col(children=[Select(dense=True, items=[227, 482, 546, 1186], label=…

In [6]:
init_source = None
def set_source(widget, event, data):
    global init_source
    init_source = data
    print('init source', init_source)

print('source', init_source)

source None


In [7]:
class Event(v.Chip):
    @out.capture()
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #self.class_ = 'px-2'
        #self.on_msg(self.handle_custom_message)
        
        #self.nr_of_clicks = 0;
#         def on_c(widget, evemt, data):
#             self.nr_of_clicks += 1
#             self.children = [f'clicks {self.nr_of_clicks}']

        self.on_event('change', update_data)

#     def handle_custom_message(self, _, content, buffers):
#         if 'type' in content.keys() and content['type'] == 'custom':
#             self.children=[f'{content["time"]}']

Event(_metadata={'mount_id': 'init_target'}, children=[f'target: {init_source}'])


Event(children=['target: None'])